# Run a Spark Job on AzureML

This notebook provides an example of how to define and run a job on AzureML using Spark. This notebook is the _control plane_, meaning it creates a connection to the AzureML workspace, defines the job, and submits the job.

**This Jupyter notebook should be run from within a compute instance on AzureML, in a Python kernel, specifically `Python 3.10 - SDK v2 (Python 3.10.11)`**. 

As you can see from the files contained in this `job` subdirectory, there are several files:

- A parametrized Python script with pyspark code that is submitted to a Spark cluster


## Create a client connection to the AzureML workspace

The following cell creates a connection object called `azureml_client` which has a connection to the AzureML workspace.

In [1]:
from azure.ai.ml import MLClient, spark, Input, Output
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import UserIdentityConfiguration

azureml_client = MLClient.from_config(
    DefaultAzureCredential(),
)


Found the config file in: /config.json


## Define the Job

The following cell defines the job. It is an object of [Spark Class](https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.spark?view=azure-python) that contains the required information to run a job:

- The cluster size
- The script to run
- The parameters for the script

In the example below, we are using the `pyspark-script-job.py` script which is parametrized. As you can see, the parameters are the following:

- `platform`: which can be `azureml` or `sagemaker`. This script is designed to be able to run on both platforms.
- `input_object_store_base_url`: Here you will use a base URL of the `s3://<BUCKETNAME>/` form for Sagemaker, or `wasbs://<CONTAINER-NAME>@<STORAGE-ACCOUNT>.blob.core.windows.net/` or `azureml://datastores/workspaceblobstore/paths/` for AzureML. **Don't forget the trailing slash /.**
- `input_path`: The path to read from
- `output_object_store_base_url`: Here you will use a base URL of the `s3://<BUCKETNAME>/` form for Sagemaker, or `wasbs://<CONTAINER-NAME>@<STORAGE-ACCOUNT>.blob.core.windows.net/` or `azureml://datastores/workspaceblobstore/paths/` for AzureML. **Don't forget the trailing slash /.**
- `output_path`: The path to write to
- `subreddits`: a comma separated string of subreddit names

This job receives the object store location for the raw data, in this case a single month. Then the job filters the original data and writes the filtered data out. This is designed to be used for either submissions or comments, not both.

For more information about the parameters used in the job definition, [read the documentation](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-submit-spark-jobs?view=azureml-api-2&tabs=sdk#submit-a-standalone-spark-job).



In [2]:
spark_job = spark(
    name="reddit-filter-job-sdk-redone",
    code="./",
    entry={"file": "pyspark-script-job.py"},
    driver_cores=1,
    driver_memory="7g",
    executor_cores=4,
    executor_memory="7g",
    executor_instances=6,
    resources={
        "instance_type": "Standard_E4S_V3",
        "runtime_version": "3.2.0",
    },
    inputs={
        "platform": "azureml",
        "input_object_store_base_url": "wasbs://bigdata@marckvnonprodblob.blob.core.windows.net/",
        "input_path": "reddit-parquet/submissions/year=*/month=*/",
        "output_object_store_base_url": "azureml://datastores/workspaceblobstore/paths/",
        "output_path": "filtered-submissions-all2",
        "subreddits": "washingtondc,Seattle,nyc,Atlanta"
    },
    identity=UserIdentityConfiguration(),
    args="--platform ${{inputs.platform}} --input_object_store_base_url ${{inputs.input_object_store_base_url}} --input_path ${{inputs.input_path}} --output_object_store_base_url ${{inputs.output_object_store_base_url}} --output_path ${{inputs.output_path}} --subreddits ${{inputs.subreddits}}"
)



## Submit the job

The following cell takes the job you defined above and submits it. If you are submitted multiple jobs, you may want to create a separate job definition object.

In [3]:
job_object = azureml_client.jobs.create_or_update(spark_job)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading code (0.48 MBs): 100%|██████

## Get the Job Studio URL

Once you submit the job, you can navigate to it in the AzureML Studio and monitor it's progress. There are ways to do it through the SDK but for now just use the Studio. These are unattended jobs, which means you can shut down this notebook and the Compute Instance, but the job will go through it's lifecycle:

- Job is submitted
- Job is queued
- Job is run
- Job completes (assuming no errors)

**Each job's Studio URL will be different.**

In [4]:
studio_url = job_object.studio_url
print(studio_url)

https://ml.azure.com/runs/reddit-filter-job-sdk-redone?wsid=/subscriptions/58bb8a15-5d27-4d02-a5ca-772d24ae37a8/resourcegroups/project-rg/workspaces/group-02-aml&tid=fd571193-38cb-437b-bb55-60f28d67b643
